<a href="https://colab.research.google.com/github/Chenchu-906/Information-Retrieval-/blob/main/Digital_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VECTOR SPACE MODEL 

Names: 

Chenchu Aravind 20MIA1126

Shiva Sindhu Perla 20MIA1104


In [15]:
""" Importing the libraries"""
import glob
import math
import re
import sys
from collections import defaultdict
from functools import reduce
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
STOPWORDS = set(stopwords.words("english"))

"""Importing the text documents"""
CORPUS = "/content/sample_data/*"

# Each document has an id, and these are the keys in the following dict.
document_filenames = dict()

# The size of the corpus
N = 0
# vocabulary: a set to contain all unique terms (i.e. words) in the corpus
vocabulary = set()
postings = defaultdict(dict)
document_frequency = defaultdict(int)
length = defaultdict(float)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
def main():
    # Get details about corpus
    get_corpus()

    # Initialise terms and postings for the corpus
    initialize_terms_and_postings()
    initialize_document_frequencies()
    initialize_lengths()
    while True:

        # Gets the sorted list of ranked documents
        scores = do_search()
        print_scores(scores)

In [17]:
def get_corpus():
    global document_filenames, N

    # Fetch list of document names in corpus
    documents = glob.glob(CORPUS)

    # Set size of corpus
    N = len(documents)

    # Dictionary having doc id as key and document name as value
    document_filenames = dict(zip(range(N), documents))

In [18]:
def initialize_terms_and_postings():
    """Reads in each document in document_filenames, splits it into a
    list of terms (i.e., tokenizes it)."""
    global vocabulary, postings
    for id in document_filenames:

        # Read the document
        with open(document_filenames[id], "r") as f:
            document = f.read()

        # Remove all special characters from the document
        document = remove_special_characters(document)

        # Remove digits from the document
        document = remove_digits(document)

        # Tokenize the document
        terms = tokenize(document)

        # Remove duplicates from the terms
        unique_terms = set(terms)

        # Add unique terms to the vocabulary
        vocabulary = vocabulary.union(unique_terms)

        # For every unique term
        for term in unique_terms:

            # The value is the frequency of the term in the document
            postings[term][id] = terms.count(term)


In [28]:
def tokenize(document):
    """Returns a list whose elements are the separate terms in document"""
    # Tokenize text into terms
    terms = word_tokenize(document)
    terms = [term.lower() for term in terms if term not in STOPWORDS] 
    return terms
document='Every year Maha Shivratri is celebrated with a lot of pomp and grandeur.'
print(tokenize(document))


['every', 'year', 'maha', 'shivratri', 'celebrated', 'lot', 'pomp', 'grandeur', '.']


In [ ]:
print()

In [29]:
def initialize_document_frequencies():
    """For each term in the vocabulary, count the number of documents
    it appears in, and store the value in document_frequncy[term]
    """
    global document_frequency
    for term in vocabulary:
        document_frequency[term] = len(postings[term])


In [30]:
def initialize_lengths():
    """ Computes the length for each document """
    global length
    for id in document_filenames:
        l = 0
        for term in vocabulary:
            l += term_frequency(term, id) ** 2
        length[id] = math.sqrt(l)

In [31]:
def term_frequency(term, id):
    """Returns the term frequency of term in document id"""
    if id in postings[term]:
        return postings[term][id]
    else:
        return 0.0

In [33]:
def inverse_document_frequency(term):
    """Returns the inverse document frequency of term."""
    if term in vocabulary:
        return math.log(N / document_frequency[term], 2)
    else:
        return 0.0
print(inverse_document_frequency(23.3))

0.0


In [36]:
def print_scores(scores):
    print("Top 3 Documents based on Cosine similarity")
    print("-" * 42)
    print("| %s | %-30s |" % ("Score", "Document"))
    print("-" * 42)

    for (id, score) in scores:
        if score != 0.0:
            print("| %s | %-30s |" % (str(score)[:5], document_filenames[id]))

    print("-" * 42, end="\n\n")
    print("Vocabulary of words:",vocabulary)


def do_search():
    """Gets the input from the user what they would like to search for, and returns a
    list of relevant documents, in decreasing order of cosine similarity
    """
    query = tokenize(input("Search query >> "))

    # Exit if query is empty
    if query == []:
        sys.exit()

    scores = sorted(
        [(id, similarity(query, id)) for id in range(N)],
        key=lambda x: x[1],
        reverse=True,
    )

    return scores


def intersection(sets):
    """Returns the intersection of all sets in the list sets."""
    return reduce(set.intersection, [s for s in sets])


In [37]:
def similarity(query, id):
    """Returns the cosine similarity between query and document id."""
    similarity = 0.0

    for term in query:

        if term in vocabulary:

            # For every term in query which is also in vocabulary,
            # calculate tf-idf score of the term and add to similarity
            similarity += term_frequency(term, id) * inverse_document_frequency(term)

    similarity = similarity / length[id]

    return similarity


def remove_special_characters(text):
    """ Removes special characters using regex substitution """
    regex = re.compile(r"[^a-zA-Z0-9\s]")
    return re.sub(regex, "", text)


def remove_digits(text):
    """ Removes digits using regex substitution """
    regex = re.compile(r"\d")
    return re.sub(regex, "", text)


if __name__ == "__main__":
    main()

Search query >> Maha Shivratri will be celebrated on February 18
Top 3 Documents based on Cosine similarity
------------------------------------------
| Score | Document                       |
------------------------------------------
| 0.272 | /content/sample_data/Doc2.txt  |
| 0.223 | /content/sample_data/Doc4.txt  |
| 0.079 | /content/sample_data/Doc1.txt  |
------------------------------------------

Vocabulary of words: {'sdc', 'devotees', 'february', 'awake', 'dances', 'important', 'auspicious', 'glee', 'significance', 'prosperity', 'one', 'mantras', 'festival', 'millions', 'stay', 'maha', 'power', 'etc', 'grandness', 'fast', 'year', 'checnhu', 'holds', 'hope', 'protects', 'grandeur', 'keep', 'people', 'blessings', 'hindu', 'lot', 'happiness', 'beautiful', 'negative', 'lord', 'every', 'momentous', 'chants', 'powerful', 'india', 'occasion', 'night', 'pomp', 'since', 'festivals', 'spirits', 'shivratri', 'fervour', 'time', 'evil', 'considered', 'prayers', 'special', 'shiva', 'this

SystemExit: ignored

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [38]:
def term_frequency(term, id):
    """Returns the term frequency of term in document id.  """
    if id in postings[term]:
        print(postings[term][id])
    else:
        return 0.0

In [2]:
doc1 = "Every year Maha Shivratri is celebrated with a lot of pomp and grandeur. It is considered to be a very special time of the year since millions of people celebrate this momentous occasion with a lot of fervour and glee."
doc2 = "Lord Shiva devotees celebrate this occasion with a lot of grandness. It is accompanied by folk dances, songs, prayers, chants, mantras etc. This year, the beautiful occasion of Maha Shivratri will be celebrated on February 18."
doc3 = "People keep a fast on this Maha shivratri, stay awake at night and pray to the lord for blessings, happiness, hope and prosperity. This festival holds a lot of significance and is considered to be one of the most important festivals in India."
doc4 = "The festival of Maha Shivratri will be celebrated on February 18 and is a very auspicious festival. This Hindu festival celebrates the power of Lord Shiva. Lord Shiva protects his devotees from negative and evil spirits. He is the epitome of powerful and auspicious energy."
     


In [3]:
def Jaccard_Similarity(doc1, doc2): 
    # List the unique words in a document
    words_doc1 = set(doc1.lower().split()) 
    words_doc2 = set(doc2.lower().split()) 
    # Find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)
    # Find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)    
    return float(len(intersection)) / len(union)

In [4]:
Jaccard_Similarity(doc1,doc2)

0.2857142857142857

In [5]:
Jaccard_Similarity(doc2,doc3)

0.1694915254237288

In [6]:
Jaccard_Similarity(doc3,doc4)

0.18333333333333332